In [5]:
# Import required libraries
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd

ModuleNotFoundError: No module named 'yfinance'

In [ ]:
# Define tickers from different sectors
tickers = ['TSLA', 'AAPL', 'JNJ']  # Example tickers: Tesla (Automotive), Apple (Technology), Johnson & Johnson (Healthcare)
exp_date = '2024-12-20'  # Set an expiration date in the future for available options
strike_price = 150  # Modify based on the dataset to check various options

In [ ]:
# Dictionary to store data for each ticker
options_data = {}

# Loop through each ticker to fetch options data
for ticker in tickers:
    stock = yf.Ticker(ticker)
    options = stock.option_chain(exp_date)
    call_option = options.calls[options.calls['strike'] == strike_price].iloc[0]
    put_option = options.puts[options.puts['strike'] == strike_price].iloc[0]
    S0 = stock.history(period='1d')['Close'][-1]  # Current stock price
    sigma = stock.history(period='1y')['Close'].pct_change().std() * np.sqrt(252)  # Historical volatility
    options_data[ticker] = {
        'stock': stock,
        'call_option': call_option,
        'put_option': put_option,
        'S0': S0,
        'sigma': sigma
    }

In [ ]:
# Define common parameters
K = strike_price  # Strike price
T = (pd.to_datetime(exp_date) - pd.to_datetime('today')).days / 365  # Time to expiration in years
r = 0.05  # Risk-free interest rate (annualized)

In [ ]:
# Monte Carlo Simulation for Asian Option
def monte_carlo_asian_option(S0, K, T, r, sigma, simulations=10000, steps=252):
    dt = T / steps
    prices = np.zeros((simulations, steps))
    prices[:, 0] = S0

    # Simulate price paths
    for t in range(1, steps):
        z = np.random.standard_normal(simulations)
        prices[:, t] = prices[:, t-1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * z)

    # Calculate average price at maturity
    avg_price = prices.mean(axis=1)
    payoffs = np.maximum(avg_price - K, 0)  # For call option
    option_price = np.exp(-r * T) * payoffs.mean()
    return option_price, prices

# Run Monte Carlo simulations for each stock and plot results
for ticker, data in options_data.items():
    asian_option_price, price_paths = monte_carlo_asian_option(data['S0'], K, T, r, data['sigma'])

    # Plotting only a subset of paths for clarity
    plt.figure(figsize=(12, 6))
    num_paths_to_plot = 50  # Plot only a subset for a cleaner display

    # Plot the sampled paths
    for i in range(num_paths_to_plot):
        plt.plot(price_paths[i], lw=0.5)

    # Plot the average path
    avg_path = price_paths.mean(axis=0)
    plt.plot(avg_path, color='black', lw=2, label='Average Path')

    plt.title(f"Simulated Price Paths for Asian Option (Subset) - {ticker}")
    plt.xlabel("Days")
    plt.ylabel("Price")
    plt.legend()
    plt.show()

    print(f"{ticker} Asian Option Price (Monte Carlo): {asian_option_price}")

In [ ]:
# Vanilla Option Payoff Tables
def calculate_vanilla_payoff(S_range, K, premium, option_type='call'):
    """Calculates the payoff for Vanilla Call and Put options over a range of prices."""
    if option_type == 'call':
        payoff = np.maximum(S_range - K, 0) - premium
    elif option_type == 'put':
        payoff = np.maximum(K - S_range, 0) - premium
    return payoff

# Range of underlying prices to visualize payoffs
S_range = np.linspace(0.5 * data['S0'], 1.5 * data['S0'], 100)

# Create payoff tables for each ticker
for ticker, data in options_data.items():
    call_payoff = calculate_vanilla_payoff(S_range, K, data['call_option']['lastPrice'], 'call')
    put_payoff = calculate_vanilla_payoff(S_range, K, data['put_option']['lastPrice'], 'put')
    asian_payoff = np.maximum(S_range.mean() - K, 0)  # Example for an Asian Call

    # Store results in a DataFrame for the payoff table
    payoff_table = pd.DataFrame({
        'Underlying Price': S_range,
        'Call Payoff': call_payoff,
        'Put Payoff': put_payoff,
        'Asian Call Payoff': asian_payoff
    })

    print(f"Payoff Table for {ticker}:")
    print(payoff_table)

In [ ]:
# Risk-Return Analysis using Sharpe Ratio
expected_return = price_paths[:, -1].mean() / price_paths[:, -1].std()  # Example: Using terminal price paths
volatility = price_paths[:, -1].std()
sharpe_ratio = (expected_return - r) / volatility

print(f"Sharpe Ratio: {sharpe_ratio:.2f}")

# Plotting Vanilla and Exotic Payoffs for one of the stocks
plt.figure(figsize=(12, 6))
plt.plot(S_range, call_payoff, label='Vanilla Call Payoff', linestyle='--')
plt.plot(S_range, put_payoff, label='Vanilla Put Payoff', linestyle=':')
plt.plot(S_range, [asian_payoff] * len(S_range), label='Asian Call Payoff', linestyle='-.')

plt.xlabel('Underlying Asset Price')
plt.ylabel('Profit/Loss')
plt.title(f'Option Payoffs for Vanilla and Exotic Options - {tickers[0]}')
plt.legend()
plt.grid(True)
plt.show()